# 컨볼루션 신경망 모델을 위한 데이터 부풀리기
훈련셋이 부족하거나 훈련셋이 시험셋의 특성을 충분히 반영하지 못할 때 이 방법을 사용하면 모델의 성능 크게 향상 시킬 수 있음.<br>
케라스에서는 데이터 부풀리기를 위한 함수를 제공하기 때문에 파라미터 셋팅만으로 간단히 데이터 부풀리기 가능함.<br>
- 현실적인 문제
- 기존 모델 결과보기
- 데이터 부풀리기
- 개선 모델 결과보기

### 1 . 현실적인 문제
개발자가 만든 시험셋과 다른 사람이 만든 시험셋이 다를수도 있다. 즉 하나의 결론을 내릴 수 있다.
    
    개발자가 시험셋을 만들면 안된다.
    
하지만 어떠한 문제에서도 미래에 들어올 데이터에 대하여는 알 수가 없기 때문에 비슷한 상황에 부딪히는 경우들이 많다.<br>
먼저 도전 시험셋으로 시험한 결과를 살펴본 뒤, 한정적인 훈련셋을 이용하여 최대한 발생할 수 있는 경우를 고려하여 훈련셋을 만드는 방법인 데이터 부풀리기에 대하여 알아보겠다.

### 2. 기존 모델 결과보기

도전 시험셋인 hard_handwriting_shape/test로 테스트해보겠다.<br>
아래 코드로 시행하면 시험셋의 평가결과가 33.3%인 것을 알 수 있다. 사실상 의미없는 분류모델이다.<br> 
즉 이 모델은 훈련셋에서만 결과가 좋은 오버피팅된 모델이라고 할 수 있다.

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
np.random.seed(3)

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./hard_handwriting_shape/train', target_size=(24,24), batch_size=3, class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('./hard_handwriting_shape/test', target_size=(24,24), batch_size=3, class_mode='categorical')

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(24, 24, 3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=100,
        validation_data=test_generator,
        validation_steps=5)

print "-- Evaluate --"
scores = model.evaluate_generator(test_generator,  steps=5)
print "%s: %.2f%%" %(model.metrics_names[1], scores[1]*100)

print "-- Predict --"
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float' : lambda x: "{0:0.3f}".format(x)})
print test_generator.class_indices
print output

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/100
15/15 [==============================] - 0s 31ms/step - loss: 0.8536 - acc: 0.6667 - val_loss: 1.7573 - val_acc: 0.3333
Epoch 2/100
15/15 [==============================] - 0s 28ms/step - loss: 0.1239 - acc: 0.9778 - val_loss: 3.4448 - val_acc: 0.4000
Epoch 3/100
15/15 [==============================] - 0s 29ms/step - loss: 0.0187 - acc: 1.0000 - val_loss: 3.6558 - val_acc: 0.4000
Epoch 4/100
15/15 [==============================] - 0s 33ms/step - loss: 0.0035 - acc: 1.0000 - val_loss: 4.5435 - val_acc: 0.3333
Epoch 5/100
15/15 [==============================] - 0s 33ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 5.2832 - val_acc: 0.3333
Epoch 6/100
15/15 [==============================] - 0s 33ms/step - loss: 5.5657e-04 - acc: 1.0000 - val_loss: 5.3244 - val_acc: 0.3333
Epoch 7/100
15/15 [==============================] - 0s 31ms/step - loss: 3.1520e-04 - acc: 1.0000 - val_loss: 5.4211 - val_

### 3. 데이터 부풀리기
케라스에서는 ImageDataGenerator 함수를 통하여 부풀리기 기능을 제공한다.<br>
아래와 같은 옵션으로 데이터 부풀리기 가능하다.

    keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0.,
    width_shift_range=0.,
    height_shift_range=0.,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=K.image_data_format())
<br>

- rotation_range = 90 : 지정된 각도 범위 내에서 임의로 원본 이미지를 회전시킴. 단위는 도이며, 정수형 -> 0~90 사이에 임의로 각도로 회전
- width_shift_range = 0.1 : 지정된 수평방향 이동 범위 내에서 임의로 원본 이미지를 이동시킴. 수치는 전체 넓이의 비율(실수) ->넓이:100이면 10 픽셀내 이동
- height_shift_range = 0.1 : 지정된 수직방향 이동 범위 내에서 임의로 원본 이미지를 이동시킴. width_shift_range랑 똑같음.
- shear_range = 0.5 : 밀림 강도 범위 내에서 임의로 원본 이미지를 변형시킴. 수치 : 시계 반대방향으로 밀림강도를 라디안으로 표시. 0.5라디안 내로 변형
- zoom_range = 0.3 : 지정된 확대/축소 범위 내에서 임의로 원본 이미지를 확대/축소시킴. 수치 : 1+수치 or 1-수치 -> 0.3이면 0.7~1.3배 크기 변화
- horizontal_flip = True : 수평방향으로 뒤집기를 한다.
- vertical_flip = True : 수직방향으로 뒤집기를 한다.
<br>
아래 코드는 ImageDataGenerator()함수를 이용하여 지정된 파라미터로 원본 이미지에 대해 데이터 부풀리기를 수행한 후 그 결과를 특정 폴더에 저장하는 코드이다.

In [6]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(3)

data_aug_gen = ImageDataGenerator(rescale=1./255,
                                                  rotation_range = 10,
                                                  width_shift_range = 0.2,
                                                  height_shift_range = 0.2,
                                                  shear_range = 0.5,  
                                                  zoom_range = [0.9, 2.2],
                                                  horizontal_flip = True,
                                                  vertical_flip = True,
                                                  fill_mode='nearest')

img = load_img('./hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i=0

# 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복 횟수를 지정하여, 지정된 반복 횟수가 되면 빠져나오도록 해야한다
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='tri', save_format='png'):
    i+=1
    if i>30:
        break

위 코드로 데이터 부풀리기가 수행된 결과 이미지는 preview 안에 저장되었다.<br>
아래 코드로 실행하면 높은 정확도를 얻었다. 도전 시험셋으로 기존 모델을 시험했을 때보다 크게 개선되었다.<br>
이는 동일한 모델을 사용하면서 훈련 데이터만 부풀려서 학습을 시켰을 뿐인데 성능 향상이 일어났다.

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
np.random.seed(3)

train_datagen = ImageDataGenerator(rescale=1./255,
                                                  rotation_range = 10,
                                                  width_shift_range = 0.2,
                                                  height_shift_range = 0.2,
                                                  shear_range = 0.5,  
                                                  zoom_range = [0.9, 2.2],
                                                  horizontal_flip = True,
                                                  vertical_flip = True,
                                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory('./hard_handwriting_shape/train', target_size=(24,24), batch_size=3, class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('./hard_handwriting_shape/test', target_size=(24,24), batch_size=3, class_mode='categorical')

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(24, 24, 3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=100,
        validation_data=test_generator,
        validation_steps=5)

print "-- Evaluate --"
scores = model.evaluate_generator(test_generator,  steps=5)
print "%s: %.2f%%" %(model.metrics_names[1], scores[1]*100)

print "-- Predict --"
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float' : lambda x: "{0:0.3f}".format(x)})
print test_generator.class_indices
print output

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/100
15/15 [==============================] - 1s 35ms/step - loss: 1.2464 - acc: 0.3333 - val_loss: 1.1041 - val_acc: 0.3333
Epoch 2/100
15/15 [==============================] - 0s 25ms/step - loss: 1.0869 - acc: 0.3333 - val_loss: 1.0838 - val_acc: 0.4667
Epoch 3/100
15/15 [==============================] - 0s 31ms/step - loss: 1.0608 - acc: 0.4889 - val_loss: 1.0651 - val_acc: 0.3333
Epoch 4/100
15/15 [==============================] - 0s 30ms/step - loss: 0.9107 - acc: 0.6000 - val_loss: 1.1152 - val_acc: 0.4000
Epoch 5/100
15/15 [==============================] - 0s 31ms/step - loss: 0.9329 - acc: 0.5333 - val_loss: 1.0340 - val_acc: 0.5333
Epoch 6/100
15/15 [==============================] - 0s 30ms/step - loss: 0.8397 - acc: 0.5778 - val_loss: 1.0738 - val_acc: 0.6667
Epoch 7/100
15/15 [==============================] - 0s 32ms/step - loss: 0.7930 - acc: 0.6889 - val_loss: 1.0481 - val_acc: 0.5

15/15 [==============================] - 0s 29ms/step - loss: 0.2623 - acc: 0.9333 - val_loss: 1.7476 - val_acc: 0.6667
Epoch 63/100
15/15 [==============================] - 0s 31ms/step - loss: 0.0715 - acc: 0.9778 - val_loss: 1.2062 - val_acc: 0.7333
Epoch 64/100
15/15 [==============================] - 0s 29ms/step - loss: 0.0711 - acc: 0.9778 - val_loss: 2.0892 - val_acc: 0.7333
Epoch 65/100
15/15 [==============================] - 0s 31ms/step - loss: 0.2299 - acc: 0.9333 - val_loss: 1.7393 - val_acc: 0.7333
Epoch 66/100
15/15 [==============================] - 1s 35ms/step - loss: 0.0460 - acc: 1.0000 - val_loss: 1.8465 - val_acc: 0.7333
Epoch 67/100
15/15 [==============================] - 0s 30ms/step - loss: 0.0220 - acc: 1.0000 - val_loss: 2.1189 - val_acc: 0.7333
Epoch 68/100
15/15 [==============================] - 0s 31ms/step - loss: 0.0571 - acc: 0.9778 - val_loss: 1.9842 - val_acc: 0.8000
Epoch 69/100
15/15 [==============================] - 0s 30ms/step - loss: 0.0281 